In [3]:
pip install pyodbc


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install JayDeBeApi



Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for JayDeBeApi from https://files.pythonhosted.org/packages/ff/1f/6a627c9bd7dea13235b65fce0fff987507269d41f957c578031796f70319/JayDeBeApi-1.2.3-py3-none-any.whl.metadata
  Obtaining dependency information for JPype1 from https://files.pythonhosted.org/packages/1f/19/144f3a767b563ba5c6d4aa534ea1f3fad9a5067c3917df4458a6e1afe0ef/JPype1-1.5.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/351.5 kB ? eta -:--:--
   ------ --------------------------------- 61.4/351.5 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 351.5/351.5 kB 4.4 MB/s eta 0:00:00


In [6]:
!pip install JPype1

Defaulting to user installation because normal site-packages is not writeable


## for the normal small scale migrations 

In [4]:
    import pandas as pd
    import jaydebeapi

    # Path to the JDBC driver JAR
    jdbc_driver_jar = "C:\\Users\\HXK0531A\\Downloads\\openedge 9.jar"

    # JDBC connection parameters for Progress DB
    jdbc_driver_class = 'com.ddtek.jdbc.openedge.OpenEdgeDriver' 
    jdbc_url = 'jdbc:datadirect:openedge://sbdbrlnx02:64105;databaseName=mfgtest'
    db_username = 'odbcuser'
    db_password = 'userodbc'

    # MySQL connection parameters
    mySQL_connection_params = {
        'drivername': 'mysql+pymysql',
        'username': 'qad_brazil_user',
        'password': 'qad_brazil_2024!',
        'host': 'tow-p-solix01',
        'port': '3307',
        'database': 'Qad_Brazil',
    }

    # Connect to Progress DB using JayDeBeApi
    progress_conn = jaydebeapi.connect(jdbc_driver_class,
                                       jdbc_url,
                                       [db_username, db_password],
                                       jdbc_driver_jar)

    # MySQL engine using SQLAlchemy
    from sqlalchemy import create_engine

    MySQL_engine = create_engine(f"{mySQL_connection_params['drivername']}://"
                                 f"{mySQL_connection_params['username']}:{mySQL_connection_params['password']}@"
                                 f"{mySQL_connection_params['host']}:{mySQL_connection_params['port']}/"
                                 f"{mySQL_connection_params['database']}")

    # List of table names to transfer
    table_names_to_transfer = ['code_mstr']  # Add your table names here

    for table_name in table_names_to_transfer:
        try:
            # Ensure the correct schema and table name
            query = f'SELECT * FROM "PUB"."{table_name}"'  # Adjust schema accordingly
            progress_data = pd.read_sql(query, progress_conn)

            target_table_name = f"{table_name}_PUB" 

            # Write data to MySQL table
            progress_data.to_sql(target_table_name, MySQL_engine, index=False, if_exists='replace')

            print(f"Table '{table_name}' migrated successfully.")
        except Exception as e:
            print(f"Error migrating table '{table_name}': {e}")


    # Close the connections
    progress_conn.close()
    MySQL_engine.dispose()


C:\Users\local_HXK0531A\Temp\ipykernel_38248\3703691519.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  progress_data = pd.read_sql(query, progress_conn)


Error migrating table 'code_mstr': java.sql.SQLException: [DataDirect][OpenEdge JDBC Driver][OpenEdge] Column code_cmmt at rowid 26661205 in table PUB.code_mstr has value exceeding its max length or precision.


## for large scale migrations

In [16]:
import pandas as pd
import jaydebeapi
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import time



# Path to the JDBC driver JAR
jdbc_driver_jar = "C:\\Users\\HXK0531A\\Downloads\\openedge 9.jar"



# JDBC connection parameters for Progress DB
jdbc_driver_class = 'com.ddtek.jdbc.openedge.OpenEdgeDriver'
jdbc_url = 'jdbc:datadirect:openedge://sbdbrlnx02:64105;databaseName=mfgtest'
db_username = 'odbcuser'
db_password = 'userodbc'

# MySQL connection parameters
mySQL_connection_params = {
        'drivername': 'mysql+pymysql',
        'username': 'qad_brazil_user',
        'password': 'qad_brazil_2024!',
        'host': 'tow-p-solix01',
        'port': '3307',
        'database': 'Qad_Brazil',
    }



# MySQL connection parameters (removed fast_executemany)
MySQL_engine = create_engine(
    f"mysql+pymysql://{mySQL_connection_params['username']}:{mySQL_connection_params['password']}@"
    f"{mySQL_connection_params['host']}:{mySQL_connection_params['port']}/{mySQL_connection_params['database']}",
    poolclass=NullPool  # Disable connection pooling
)



# Function to manually fetch data from Progress DB using JayDeBeApi and convert to Pandas DataFrame
def fetch_data_in_chunks(query, conn, chunk_size=50000):
    cursor = conn.cursor()
    cursor.execute(query)

    columns = [desc[0] for desc in cursor.description]  # Get column names
    while True:
        rows = cursor.fetchmany(chunk_size)  # Fetch chunk_size rows
        if not rows:
            break
        yield pd.DataFrame(rows, columns=columns)  # Yield DataFrame chunk

        
        
# Function to migrate tables in chunks
def migrate_table_in_chunks(progress_conn, table_name, mysql_engine, chunk_size=50000):
    try:
        # Ensure the correct schema and table name
        query = f'SELECT * FROM "PUB"."{table_name}"'  
        
        # Fetch data in chunks and migrate
        target_table_name = f"{table_name}_PUB"
        for chunk_num, chunk in enumerate(fetch_data_in_chunks(query, progress_conn, chunk_size)):
            # Write chunk to MySQL table
            chunk.to_sql(target_table_name, mysql_engine, index=False, if_exists='append')
            print(f"Chunk {chunk_num + 1} of table '{table_name}' migrated successfully.")
        
        print(f"Table '{table_name}' migration completed successfully.")
    except Exception as e:
        print(f"Error migrating table '{table_name}': {e}")

        
        
        
# Connect to Progress DB using JayDeBeApi
progress_conn = None
try:
    start_time = time.time()
    
    progress_conn = jaydebeapi.connect(jdbc_driver_class, jdbc_url, [db_username, db_password], jdbc_driver_jar)
    
    # List of table names to transfer
    table_names_to_transfer = ['lgd_mstr']  # Add your table names here

    for table_name in table_names_to_transfer:
        migrate_table_in_chunks(progress_conn, table_name, MySQL_engine)

    end_time = time.time()
    print(f"Data migration completed in {end_time - start_time:.2f} seconds.")
    
finally:
    # Ensure that connections are closed properly
    if progress_conn:
        progress_conn.close()
    MySQL_engine.dispose()
    print("All connections closed.")


Error migrating table 'lgd_mstr': java.sql.SQLException: [DataDirect][OpenEdge JDBC Driver][OpenEdge] Column lgd_doc_type at rowid 252673 in table PUB.lgd_mstr has value exceeding its max length or precision.
Data migration completed in 0.85 seconds.
All connections closed.


## for black database 

In [1]:
import pandas as pd
import jaydebeapi
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import time

# Path to the JDBC driver JAR
jdbc_driver_jar = "C:\\Users\\HXK0531A\\Downloads\\openedge 9.jar"

# JDBC connection parameters for Progress DB
jdbc_driver_class = 'com.ddtek.jdbc.openedge.OpenEdgeDriver'
jdbc_url = 'jdbc:datadirect:openedge://sbdbrlnx02:64170;databaseName=black'
db_username = 'odbcuser'
db_password = 'userodbc'

# MySQL connection parameters
mySQL_connection_params = {
        'drivername': 'mysql+pymysql',
        'username': 'qad_brazil_user',
        'password': 'qad_brazil_2024!',
        'host': 'tow-p-solix01',
        'port': '3307',
        'database': 'Qad_Brazil',
    }

# MySQL connection parameters (removed fast_executemany)
MySQL_engine = create_engine(
    f"mysql+pymysql://{mySQL_connection_params['username']}:{mySQL_connection_params['password']}@"
    f"{mySQL_connection_params['host']}:{mySQL_connection_params['port']}/{mySQL_connection_params['database']}",
    poolclass=NullPool  # Disable connection pooling
)

# Function to manually fetch data from Progress DB using JayDeBeApi and convert to Pandas DataFrame
def fetch_data_in_chunks(query, conn, chunk_size=50000):
    cursor = conn.cursor()
    cursor.execute(query)

    columns = [desc[0] for desc in cursor.description]  # Get column names
    while True:
        rows = cursor.fetchmany(chunk_size)  # Fetch chunk_size rows
        if not rows:
            break
        yield pd.DataFrame(rows, columns=columns)  # Yield DataFrame chunk

# Function to migrate tables in chunks
def migrate_table_in_chunks(progress_conn, table_name, mysql_engine, chunk_size=50000):
    try:
        # Ensure the correct schema and table name
        query = f'SELECT * FROM "PUB"."{table_name}"'  
        
        # Fetch data in chunks and migrate
        target_table_name = f"{table_name}_pub"
        for chunk_num, chunk in enumerate(fetch_data_in_chunks(query, progress_conn, chunk_size)):
            # Write chunk to MySQL table
            chunk.to_sql(target_table_name, mysql_engine, index=False, if_exists='append')
            print(f"Chunk {chunk_num + 1} of table '{table_name}' migrated successfully.")
        
        print(f"Table '{table_name}' migration completed successfully.")
    except Exception as e:
        print(f"Error migrating table '{table_name}': {e}")

# Connect to Progress DB using JayDeBeApi
progress_conn = None
try:
    start_time = time.time()
    
    progress_conn = jaydebeapi.connect(jdbc_driver_class, jdbc_url, [db_username, db_password], jdbc_driver_jar)
    
    # List of table names to transfer
    table_names_to_transfer = ['bd_ld_det','fotoped','zpcs_det','movimento-parceria','zpxr_det']  # Add your table names here

    for table_name in table_names_to_transfer:
        migrate_table_in_chunks(progress_conn, table_name, MySQL_engine)

    end_time = time.time()
    print(f"Data migration completed in {end_time - start_time:.2f} seconds.")
    
finally:
    # Ensure that connections are closed properly
    if progress_conn:
        progress_conn.close()
    MySQL_engine.dispose()
    print("All connections closed.")


Error migrating table 'bd_ld_det': java.sql.SQLException: [DataDirect][OpenEdge JDBC Driver][OpenEdge] Column bd_ld_user2 at rowid 35229865 in table PUB.bd_ld_det has value exceeding its max length or precision.
Chunk 1 of table 'fotoped' migrated successfully.
Chunk 2 of table 'fotoped' migrated successfully.
Chunk 3 of table 'fotoped' migrated successfully.
Chunk 4 of table 'fotoped' migrated successfully.
Chunk 5 of table 'fotoped' migrated successfully.
Chunk 6 of table 'fotoped' migrated successfully.
Chunk 7 of table 'fotoped' migrated successfully.
Chunk 8 of table 'fotoped' migrated successfully.
Chunk 9 of table 'fotoped' migrated successfully.
Chunk 10 of table 'fotoped' migrated successfully.
Chunk 11 of table 'fotoped' migrated successfully.
Chunk 12 of table 'fotoped' migrated successfully.
Chunk 13 of table 'fotoped' migrated successfully.
Chunk 14 of table 'fotoped' migrated successfully.
Chunk 15 of table 'fotoped' migrated successfully.
Chunk 16 of table 'fotoped' migr

Chunk 157 of table 'fotoped' migrated successfully.
Chunk 158 of table 'fotoped' migrated successfully.
Chunk 159 of table 'fotoped' migrated successfully.
Chunk 160 of table 'fotoped' migrated successfully.
Chunk 161 of table 'fotoped' migrated successfully.
Chunk 162 of table 'fotoped' migrated successfully.
Chunk 163 of table 'fotoped' migrated successfully.
Chunk 164 of table 'fotoped' migrated successfully.
Chunk 165 of table 'fotoped' migrated successfully.
Chunk 166 of table 'fotoped' migrated successfully.
Chunk 167 of table 'fotoped' migrated successfully.
Chunk 168 of table 'fotoped' migrated successfully.
Chunk 169 of table 'fotoped' migrated successfully.
Chunk 170 of table 'fotoped' migrated successfully.
Chunk 171 of table 'fotoped' migrated successfully.
Chunk 172 of table 'fotoped' migrated successfully.
Chunk 173 of table 'fotoped' migrated successfully.
Chunk 174 of table 'fotoped' migrated successfully.
Chunk 175 of table 'fotoped' migrated successfully.
Chunk 176 of

Chunk 5 of table 'zpcs_det' migrated successfully.
Chunk 6 of table 'zpcs_det' migrated successfully.
Chunk 7 of table 'zpcs_det' migrated successfully.
Chunk 8 of table 'zpcs_det' migrated successfully.
Chunk 9 of table 'zpcs_det' migrated successfully.
Chunk 10 of table 'zpcs_det' migrated successfully.
Chunk 11 of table 'zpcs_det' migrated successfully.
Chunk 12 of table 'zpcs_det' migrated successfully.
Chunk 13 of table 'zpcs_det' migrated successfully.
Chunk 14 of table 'zpcs_det' migrated successfully.
Chunk 15 of table 'zpcs_det' migrated successfully.
Chunk 16 of table 'zpcs_det' migrated successfully.
Chunk 17 of table 'zpcs_det' migrated successfully.
Chunk 18 of table 'zpcs_det' migrated successfully.
Chunk 19 of table 'zpcs_det' migrated successfully.
Chunk 20 of table 'zpcs_det' migrated successfully.
Chunk 21 of table 'zpcs_det' migrated successfully.
Chunk 22 of table 'zpcs_det' migrated successfully.
Chunk 23 of table 'zpcs_det' migrated successfully.
Chunk 24 of table

Chunk 162 of table 'zpcs_det' migrated successfully.
Chunk 163 of table 'zpcs_det' migrated successfully.
Chunk 164 of table 'zpcs_det' migrated successfully.
Chunk 165 of table 'zpcs_det' migrated successfully.
Chunk 166 of table 'zpcs_det' migrated successfully.
Chunk 167 of table 'zpcs_det' migrated successfully.
Chunk 168 of table 'zpcs_det' migrated successfully.
Chunk 169 of table 'zpcs_det' migrated successfully.
Chunk 170 of table 'zpcs_det' migrated successfully.
Chunk 171 of table 'zpcs_det' migrated successfully.
Chunk 172 of table 'zpcs_det' migrated successfully.
Chunk 173 of table 'zpcs_det' migrated successfully.
Chunk 174 of table 'zpcs_det' migrated successfully.
Chunk 175 of table 'zpcs_det' migrated successfully.
Chunk 176 of table 'zpcs_det' migrated successfully.
Chunk 177 of table 'zpcs_det' migrated successfully.
Chunk 178 of table 'zpcs_det' migrated successfully.
Chunk 179 of table 'zpcs_det' migrated successfully.
Chunk 180 of table 'zpcs_det' migrated success

Chunk 317 of table 'zpcs_det' migrated successfully.
Chunk 318 of table 'zpcs_det' migrated successfully.
Chunk 319 of table 'zpcs_det' migrated successfully.
Chunk 320 of table 'zpcs_det' migrated successfully.
Chunk 321 of table 'zpcs_det' migrated successfully.
Chunk 322 of table 'zpcs_det' migrated successfully.
Chunk 323 of table 'zpcs_det' migrated successfully.
Chunk 324 of table 'zpcs_det' migrated successfully.
Chunk 325 of table 'zpcs_det' migrated successfully.
Chunk 326 of table 'zpcs_det' migrated successfully.
Chunk 327 of table 'zpcs_det' migrated successfully.
Chunk 328 of table 'zpcs_det' migrated successfully.
Chunk 329 of table 'zpcs_det' migrated successfully.
Chunk 330 of table 'zpcs_det' migrated successfully.
Chunk 331 of table 'zpcs_det' migrated successfully.
Chunk 332 of table 'zpcs_det' migrated successfully.
Chunk 333 of table 'zpcs_det' migrated successfully.
Chunk 334 of table 'zpcs_det' migrated successfully.
Chunk 335 of table 'zpcs_det' migrated success

Chunk 472 of table 'zpcs_det' migrated successfully.
Chunk 473 of table 'zpcs_det' migrated successfully.
Chunk 474 of table 'zpcs_det' migrated successfully.
Chunk 475 of table 'zpcs_det' migrated successfully.
Chunk 476 of table 'zpcs_det' migrated successfully.
Chunk 477 of table 'zpcs_det' migrated successfully.
Chunk 478 of table 'zpcs_det' migrated successfully.
Chunk 479 of table 'zpcs_det' migrated successfully.
Chunk 480 of table 'zpcs_det' migrated successfully.
Chunk 481 of table 'zpcs_det' migrated successfully.
Chunk 482 of table 'zpcs_det' migrated successfully.
Chunk 483 of table 'zpcs_det' migrated successfully.
Chunk 484 of table 'zpcs_det' migrated successfully.
Chunk 485 of table 'zpcs_det' migrated successfully.
Chunk 486 of table 'zpcs_det' migrated successfully.
Chunk 487 of table 'zpcs_det' migrated successfully.
Chunk 488 of table 'zpcs_det' migrated successfully.
Chunk 489 of table 'zpcs_det' migrated successfully.
Chunk 490 of table 'zpcs_det' migrated success

KeyboardInterrupt: 

## increase of length (space) in the target for the columns to be migrated 

In [ ]:
import pandas as pd
import jaydebeapi
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import time

# Path to the JDBC driver JAR
jdbc_driver_jar = "C:\\Users\\HXK0531A\\Downloads\\openedge 9.jar"

# JDBC connection parameters for Progress DB
jdbc_driver_class = 'com.ddtek.jdbc.openedge.OpenEdgeDriver'
jdbc_url = 'jdbc:datadirect:openedge://sbdbrlnx02:64105;databaseName=mfgtest'
db_username = 'odbcuser'
db_password = 'userodbc'

# MySQL connection parameters
mySQL_connection_params = {
    'drivername': 'mysql+pymysql',
    'username': 'qad_brazil_user',
    'password': 'qad_brazil_2024!',
    'host': 'tow-p-solix01',
    'port': '3307',
    'database': 'Qad_Brazil',
}

# MySQL engine setup
MySQL_engine = create_engine(
    f"mysql+pymysql://{mySQL_connection_params['username']}:{mySQL_connection_params['password']}@"
    f"{mySQL_connection_params['host']}:{mySQL_connection_params['port']}/{mySQL_connection_params['database']}",
    poolclass=NullPool  # Disable connection pooling
)

# Function to fetch data excluding problematic columns
def fetch_data_with_selected_columns(table_name, columns_to_exclude, conn):
    # Get all columns excluding problematic ones
    cursor = conn.cursor()
    cursor.execute(f'SELECT * FROM "PUB"."{table_name}" WHERE 1=0')  # Get column names without fetching data
    all_columns = [desc[0] for desc in cursor.description]

    # Remove the columns to exclude
    selected_columns = [col for col in all_columns if col not in columns_to_exclude]

    # Create a string of columns for the SELECT query
    columns_string = ', '.join([f'"{col}"' for col in selected_columns])

    # Fetch the data using the selected columns
    query = f'SELECT {columns_string} FROM "PUB"."{table_name}"'
    df = pd.read_sql(query, conn)

    print(f"Fetched {len(df)} records from Progress table '{table_name}' excluding columns {columns_to_exclude}.")
    return df

# Function to migrate the data
def migrate_table(progress_conn, table_name, mysql_engine, columns_to_exclude):
    try:
        df = fetch_data_with_selected_columns(table_name, columns_to_exclude, progress_conn)
        target_table_name = f"{table_name}_PUB"
        
        # Write DataFrame to MySQL, replacing if it exists
        df.to_sql(target_table_name, mysql_engine, index=False, if_exists='replace')
        print(f"Data migrated successfully to MySQL table '{target_table_name}'.")

    except Exception as e:
        print(f"Error migrating table '{table_name}': {e}")

# Main execution block
progress_conn = None
try:
    start_time = time.time()
    
    progress_conn = jaydebeapi.connect(jdbc_driver_class, jdbc_url, [db_username, db_password], jdbc_driver_jar)
    
    # Specify the table names and the columns to exclude
    table_names_to_transfer = ['lgd_mstr']  # Add your table names here
    columns_to_exclude = ["lgd_doc_type", "lgd_user2", "lgd_user1", "lgd_prim_doc_typ", "lgd_domain","lgd_rmks","lgd_shipto_site"]  # Add any other problematic columns here

    for table_name in table_names_to_transfer:
        migrate_table(progress_conn, table_name, MySQL_engine, columns_to_exclude)

    end_time = time.time()
    print(f"Data migration completed in {end_time - start_time:.2f} seconds.")
    
finally:
    if progress_conn:
        progress_conn.close()
    MySQL_engine.dispose()
    print("All connections closed.")


C:\Users\local_HXK0531A\Temp\ipykernel_94700\2687913043.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
